In [1]:
import pandas as pd
import os
import logging

In [2]:
logging.basicConfig(level=logging.INFO, format=' %(asctime)s - %(levelname)s - %(message)s')

In [3]:
baojiadan_folder = 'D:\\Program Files (x86)\\百度云同步盘\\Dropbox\\-E·J- 2014.5.1\\2015.9.26 燕文\\'

In [4]:
baojiadan_file_list = list()

In [5]:
for file_name in os.listdir(baojiadan_folder):
    if file_name.startswith('广州燕文亚马逊报价单'):
        baojiadan_file_list.append(file_name)

In [6]:
latest_baojiadan_file_name = baojiadan_file_list[-1]
# latest_baojiadan_file_name

In [7]:
latest_baojiadan_path = os.path.join(baojiadan_folder, latest_baojiadan_file_name)
# latest_baojiadan_path

In [8]:
baojiadan_sheets_dict = pd.read_excel(latest_baojiadan_path, sheetname=None)
# type(baojiadan)

In [9]:
wanted_shipping_service_list = ['亚马逊燕文航空挂号小包-普货',
                                '亚马逊燕文航空挂号小包-特货',
                                '亚马逊燕邮宝挂号-特货',
                                '亚马逊燕文航空易派小包-普货',
                                '亚马逊燕文航空易派小包-特货',
                                '亚马逊燕文专线追踪小包-普货',
                                '亚马逊燕文专线追踪小包-特货'
]

In [10]:
export_path = os.path.join(baojiadan_folder, '报价单','燕文（amazon）报价表格.xlsx')
writer = pd.ExcelWriter(export_path)

pd_to_merge = []

for service_name in wanted_shipping_service_list:
    if service_name in baojiadan_sheets_dict.keys():
        logging.info(service_name + ' is ready to export.')
        df = baojiadan_sheets_dict[service_name]
        try:
            for c in df.columns:
                for index, r in enumerate(df[c]):
                    if r == '国家':
                        country_cell_location = [index, c]
#                         print(country_cell_location)
                    if r == '价格使用说明:':
                        end_location = [index, c]
#                         print(country_cell_location)
                        break
            df = df[country_cell_location[0]+1:end_location[0]]
            df = df.dropna(how='all')
            df = df[df[country_cell_location[1]].str.len() < 10]
            df['service_name'] = service_name
            print(df.head(5))
            
#             tiers = [1, 2, 3]
#             for tier in tiers:
#                 this_c_name = 'tier_' + str(tier)
#                 next_c = tier + 1
#                 try:
#                     df[this_c_name + '_base'] = df[[tier]]
#                     df[this_c_name + '_surcharge'] = df[[next_c]]
#                     df[this_c_name + '_total'] = df[[tier]].astype(float) * 0.35 + df[this_c_name + '_surcharge'].astype(float)
#                 except Exception as err:
#                     print(str(err))
            
            kg = 0.35
            tiers = [1, 2, 3]
            for tier in tiers:
                this_c_name = 'tier_' + str(tier)
                column_num = 2 * tier - 1
                try:
                    df[this_c_name + '_kg_fee'] = df[[column_num]] * kg
                    df[this_c_name + '_kg_fee'] = df[this_c_name + '_kg_fee'].astype(float)
                    df[this_c_name + '_surcharge'] = df[[column_num + 1]].astype(float)
                    df[this_c_name + '_total'] = df[this_c_name + '_kg_fee'] + df[this_c_name + '_surcharge']
                except Exception as err:
                    print(str(err))
                    
#             df_for_merge = df[['service_name', 0, 'tier_1_total']]
            
#             pd_to_merge.append(df_for_merge)
    
            df.to_excel(writer, sheet_name=service_name, index=False)
            logging.info(service_name + ' exported.')
            # df.to_csv(os.path.join(baojiadan_folder, '报价单', key + '.csv'))
        except Exception as err:
            logging.info(service_name + ' failed to export. ' + str(err))
            pass
        # df.to_csv(os.path.join(baojiadan_folder, '报价单', key + '.xlsx'))
        
writer.save()

 2020-07-01 23:41:45,086 - INFO - 亚马逊燕文航空挂号小包-普货 is ready to export.
 2020-07-01 23:41:45,208 - INFO - 亚马逊燕文航空挂号小包-普货 exported.
 2020-07-01 23:41:45,209 - INFO - 亚马逊燕文航空挂号小包-特货 is ready to export.
 2020-07-01 23:41:45,321 - INFO - 亚马逊燕文航空挂号小包-特货 exported.
 2020-07-01 23:41:45,322 - INFO - 亚马逊燕邮宝挂号-特货 is ready to export.


  亚马逊燕文航空挂号小包-普货 Unnamed: 1 Unnamed: 2 Unnamed: 3 Unnamed: 4 Unnamed: 5  \
4             美国        100         25        100         25        100   
5            加拿大        130         24        130         24        130   
6             英国         58         17         58         17         58   
7             法国         80         17         80         17         80   
8             德国         70         17         70         17         70   

  Unnamed: 6  返回主目录    service_name  
4         25    NaN  亚马逊燕文航空挂号小包-普货  
5         24    NaN  亚马逊燕文航空挂号小包-普货  
6         17    NaN  亚马逊燕文航空挂号小包-普货  
7         17    NaN  亚马逊燕文航空挂号小包-普货  
8         17    NaN  亚马逊燕文航空挂号小包-普货  
  亚马逊燕文航空挂号小包-特货 Unnamed: 1 Unnamed: 2 Unnamed: 3 Unnamed: 4 Unnamed: 5  \
4             美国        107         28        107         28        107   
5            加拿大        130         24        130         24        130   
6             英国         68         17         68         17         68   
7             法国     

 2020-07-01 23:41:45,376 - INFO - 亚马逊燕邮宝挂号-特货 exported.
 2020-07-01 23:41:45,377 - INFO - 亚马逊燕文航空易派小包-普货 is ready to export.
 2020-07-01 23:41:45,440 - INFO - 亚马逊燕文航空易派小包-普货 exported.
 2020-07-01 23:41:45,440 - INFO - 亚马逊燕文航空易派小包-特货 is ready to export.
 2020-07-01 23:41:45,497 - INFO - 亚马逊燕文航空易派小包-特货 exported.
 2020-07-01 23:41:45,498 - INFO - 亚马逊燕文专线追踪小包-普货 is ready to export.
 2020-07-01 23:41:45,523 - INFO - 亚马逊燕文专线追踪小包-普货 exported.
 2020-07-01 23:41:45,525 - INFO - 亚马逊燕文专线追踪小包-特货 is ready to export.
 2020-07-01 23:41:45,552 - INFO - 亚马逊燕文专线追踪小包-特货 exported.


  亚马逊燕邮宝挂号-特货 Unnamed: 1 Unnamed: 2  返回主目录 service_name
4          英国       47.5         30    NaN  亚马逊燕邮宝挂号-特货
5          法国       46.5         18    NaN  亚马逊燕邮宝挂号-特货
6          德国       47.5         21    NaN  亚马逊燕邮宝挂号-特货
7         意大利       43.5         32    NaN  亚马逊燕邮宝挂号-特货
8         西班牙       55.5         20    NaN  亚马逊燕邮宝挂号-特货
could not convert string to float: '亚马逊燕邮宝挂号-特货'
  亚马逊燕文航空易派小包-普货 Unnamed: 1 Unnamed: 2 Unnamed: 3 Unnamed: 4 Unnamed: 5  \
4             美国        100         25        100         25        100   
5            加拿大        130         24        130         24        130   
6             英国         58         17         58         17         58   
7             法国         80         17         80         17         80   
8             德国         70         17         70         17         70   

  Unnamed: 6  返回主目录    service_name  
4         25    NaN  亚马逊燕文航空易派小包-普货  
5         24    NaN  亚马逊燕文航空易派小包-普货  
6         17    NaN  亚马逊燕文航空易派小包-普货  
7         17